In [ ]:
import sys, urllib.request

IS_COLAB = 'google.colab' in sys.modules

    
if IS_COLAB:
    #create extra folders
    !mkdir -pv data images model

    # will install missing packages if running in colab
    print('Installing uproot...')
    !pip install uproot

    #will download the data if running in colab
    urllib.request.urlretrieve('https://cernbox.cern.ch/index.php/s/GRAyoFxBHAlMVpu/download',
                               'data/events_6D64x64.root')
    urllib.request.urlretrieve('https://cernbox.cern.ch/index.php/s/dZ9r1nvBRTpJMNW/download',
                               'data/events_6D64x64_ATLAS_resolution_6D64x64.root')
    print('ls data')
    !ls data/*

    #download additional modules used with the notebook
    !wget https://raw.githubusercontent.com/mpitt82/AI/master/ParticleImages/train_helpers.py
    !wget https://raw.githubusercontent.com/mpitt82/AI/master/ParticleImages/particleImages_helpers.py

    print('Using COLAB: all additional packages are installed and the test data is downloaded')

# Super resolution notebook

In this notebook, the Super Resolution (SR) will be applied on detector images. The goal is to reconstruct detector images with higher resolution. In this notebook we will try several approaches to this task.

The input data obtained using the [ATLAS-simplified](https://github.com/mpitt82/Geant4-models/tree/master/ATLAS-simplified) Geant4 simulation model. The bare data files are available [here](https://cernbox.cern.ch/index.php/s/oCg3en1GHAvYSTo?path=%2FCalo_RectangularGeo_diPion64x64%2Frun).

Using the [Vector2Matrix.cc](https://github.com/mpitt82/Geant4-models/blob/master/ATLAS-simplified/scripts/Vector2Matrix.cc) script $6\times 64\times 64$ are obtaned, then with [Matrix2Matrix6L.cc](https://github.com/mpitt82/Geant4-models/blob/master/ATLAS-simplified/scripts/Matrix2Matrix6L.cc) script Low-resolution images are computed. For democtaric upsaling (used as a first approach) the [Matrix2Matrix6L.cc](https://github.com/mpitt82/Geant4-models/blob/master/ATLAS-simplified/scripts/Matrix6L2Matrix.cc) script can be used. **NOTE** All scripts executed in [CINT](https://root.cern.ch/cint). 


Standards imports

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

### Loading the data

In this example, we will use two example files from cernbox folder, which have $\sim$2k events.

In [ ]:
inputfileHR = 'data/events_6D64x64.root'
inputfileLR = 'data/events_6D64x64_ATLAS_resolution_6D64x64.root'

f = uproot.open(inputfileHR)
treeHR = f['EventTree']

f = uproot.open(inputfileLR)
treeLR = f['EventTree']

print('Total number of events in both files is = '+str(treeHR.numentries)+' (HR) and '+str(treeLR.numentries)+' (LR)')

### Calorimeter images

In this section, we will create calorimeter images from the cell hits using LR and HR input files. First we will constract a dataset using SRDataLoader defined in [train_helpers.py](train_helpers.py)

In [ ]:
from train_helpers import SRDataLoader

In [ ]:
train_dataset = SRDataLoader(treeLR, treeHR)
test_dataset  = SRDataLoader(treeLR, treeHR)

Draw a random image from the energy matrices:

In [ ]:
from particleImages_helpers import DrawEventSR

In [ ]:
DrawEventSR(train_dataset, event_number = 200)

### Model

In the first implementation, we are using democratic interpulation, from varying resolution to a constant resolution of $6\times 64 \times 64$. The resulting image we will process through a model to recover the $HR$ image.

In [ ]:
device_number = 0; #change this value to "1" if using the second GPU
use_cuda = torch.cuda.is_available()
print('Availability of CUDA:',use_cuda)
print('Availability of CUNN:',torch.backends.cudnn.enabled)
print('Total number of GPU devices: ',torch.cuda.device_count())
device = torch.device("cuda:"+str(device_number) if torch.cuda.is_available() else "cpu")
if use_cuda:
    torch.cuda.set_device(device_number)
    idevice = torch.cuda.current_device()
    print('Will work on device number',idevice,', named: ',torch.cuda.get_device_name(idevice))
else: print('will run on CPU, using',torch.get_num_threads(),'cores')

In [ ]:
class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1,bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1,bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, (diffX // 2, diffX - diffX//2,
                        diffY // 2, diffY - diffY//2))
        
        # for padding issues, see 
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd

        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x
    
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        #self.down1 = down(64, 128)
        #self.down2 = down(128, 256)
        #self.down3 = down(256, 512)
        #self.down4 = down(512, 512)
        #self.up1 = up(1024, 256)
        #self.up2 = up(512, 128)
        #self.up3 = up(256, 64)
        #self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)

    def forward(self, x):
        #print('input.shape',x.shape)
        x1 = self.inc(x)
        #print('x1.shape',x1.shape)
        #x2 = self.down1(x1)
        #print('x2.shape',x2.shape)
        #x3 = self.down2(x2)
        #print('x3.shape',x3.shape)
        #x4 = self.down3(x3)
        #print('x4.shape',x4.shape)
        #x5 = self.down4(x4)
        #print('x5.shape',x5.shape)
        #x = self.up1(x5, x4)
        #print('(x5,x3).shape',x.shape)
        #x = self.up2(x, x3)
        #print('(x,x3).shape',x.shape)
        #x = self.up3(x, x2)
        #print('(x,x2).shape',x.shape)
        #x = self.up4(x, x1)
        #print('up4(x,x1).shape',x.shape)
        #x = self.outc(x) replace by line bellow
        x = self.outc(x1)
        #print('x.shape',x.shape)
        return F.relu(x)

In [ ]:
model = UNet(6,6)

In [ ]:
print('model parameters = ',sum(p.numel() for p in model.parameters() if p.requires_grad))

### Prepare the trainer and tester


In [ ]:
from train_helpers import trainMe, CreateCash

In [ ]:
#criterion = nn.MSELoss()
def criterion(yhat, y):
    vec = (yhat - y).pow(2)
#    print('yhat=',yhat.sum(),'y=',y.sum())
    return  torch.mean( vec ) 


In [ ]:
train_loader=DataLoader(dataset=train_dataset,batch_size=516 ) #train_size
optimizer = optim.Adam(model.parameters(), lr = 0.01)
cacheSR = CreateCash(model)

Train the model

In [ ]:
import time
from train_helpers import printProgressBar

def trainMe2(train_loader, model, optimizer, criterion, epochs=1000, cache={'loss':[]}, device=torch.device("cpu")):
    print('len of cache is = ',len(cache['loss']))
    isGPU = torch.cuda.is_available() and 'cuda'==device.type
    if isGPU and not next(model.parameters()).is_cuda:
        print('copy the model to GPU')
        model.to(device)
    tic = time.time()
    for epoch in range(epochs):
        for x, y in train_loader:
            #print('x.shape',x.shape)
            #print('y.shape',y.shape)
            x = x.reshape((x.shape[0],6,64,64))
            y = y.reshape((y.shape[0],6,64,64))
            #print('x.shape',x.shape)
            #print('y.shape',y.shape)
            if isGPU:
                if isinstance(x,type([])):
                    x = [k.to(device) for k in x]
                else: x = x.to(device)
                y = y.to(device)
            loss=criterion(model(x),y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        cache['loss'].append(loss.item())
        printProgressBar(epoch, epochs, loss.item())
    toc = time.time()
    print('total time: %2.2f sec' %(toc-tic))
    if isGPU:
        print('cached',torch.cuda.memory_cached(device)/1e6,' MB')
        torch.cuda.empty_cache() 
        print('Cache cleaned\nRemaining cached memory',torch.cuda.memory_cached(device)/1e6,' MB')
    return  cache

In [ ]:
cacheSR = trainMe2(train_loader, model, optimizer, criterion, 10, cacheSR, device)
#saving the model
torch.save({
    'model_state_dict' : model.cpu().state_dict(),
    'cache' : cacheLR,
    },
    'models/UNET_SR_dipion_dict_v1.pt')

In [ ]:
plt.plot(cacheSR['loss'])
plt.show()

# Ploting the results

Now we will plot the obtained results of the SR images. The results will be later tested in the downstream task.

In case we have newly trained model, we would like first to load the trained weights:

In [ ]:
#load model
checkpoint = torch.load('models/UNET_SR_dipion_dict_v1.pt')
model.load_state_dict(checkpoint['model_state_dict'])
cacheLR = checkpoint['cache']
model.train()
print('model loaded')

In [ ]:
def DrawEventSR2(data_generator, event_number, model = None, device = torch.device("cpu")):

    imageLR, imageHR = data_generator[event_number]
    
    nimage = 2
    Title = ['LR image', 'HR image',]
    if model:
        imageHRbar = model(torch.FloatTensor(imageLR).to(device)).cpu().detach().numpy()
        nimage = 3
        Title.append('SR image')
    
    fig, ax = plt.subplots(6, nimage, figsize=(10*nimage, 60))
    LayerNames=['ECAL1','ECAL2','ECAL3','HCAL1','HCAL2','HCAL3']

    for layer_i in range(6):

        LR_image = imageLR[0][layer_i]
        HR_image = imageHR[0][layer_i]

        ax[layer_i][0].imshow( LR_image, cmap='plasma', vmin=0.001, vmax=10.8 )
        ax[layer_i][1].imshow( HR_image, cmap='plasma', vmin=0.001, vmax=10.8 )

        major_ticks = np.arange(-0.5, 63.5, 1)
        minor_ticks = np.arange(-0.5, 63.5, 1)

        ax[layer_i][0].set_ylabel(LayerNames[layer_i],fontsize=64)
        for ipad in range(nimage) : 
            ax[layer_i][ipad].set_xticks(minor_ticks, minor=True)
            ax[layer_i][ipad].set_yticks(minor_ticks, minor=True)
            ax[layer_i][ipad].grid(which='minor')
            ax[0][ipad].set_title(Title[ipad]+' ' , fontsize=48)     
        ax[layer_i][0].text(5,5,'E = %2.2f GeV'%(LR_image.sum()/1e3),fontsize=32,bbox={'facecolor': 'white'})
        ax[layer_i][1].text(5,5,'E = %2.2f GeV'%(HR_image.sum()/1e3),fontsize=32,bbox={'facecolor': 'white'})
        
        if model:
            SR_image = imageHRbar[0][layer_i]
            ax[layer_i][2].imshow( SR_image, cmap='plasma', vmin=0.001, vmax=10.8 )
            ax[layer_i][2].text(5,5,'E = %2.2f GeV'%(SR_image.sum()/1e3),fontsize=32,bbox={'facecolor': 'white'})

    plt.tight_layout()
    fig.savefig('images/SR_ev_' + str(event_number) + '.pdf')
    fig.savefig('images/SR_ev_' + str(event_number) + '.png')
    plt.show()

In [ ]:
DrawEventSR2(train_dataset, event_number = 200, model = model, device = device)